# Initial Setup

In [ ]:
!pip install -U -q "tf-models-official"

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.python.ops import control_flow_ops

2024-05-11 07:01:38.572011: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-11 07:01:38.573709: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 07:01:38.596078: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 07:01:38.596095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 07:01:38.596770: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# LabelMaps

## Creacion de Labelmaps

In [ ]:
!python ./csv_to_labelmap.py

# TFRecords

## Train Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/train/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/train" train.record

## Test Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/test/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/test" test.record

## Valid Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/valid/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/valid" valid.record

# MLModel

## Libraries

In [2]:
import wget
import tarfile
import os
import re

## Enviroment Settings

In [3]:
#Can be configured
DATASET_DIRECTORY = "/tf/dataset/workspaceCards/"
OBJECT_DETECTION_DIRECTORY = "/tf/tensorflow/models/research/object_detection/"

MODEL_NAME = "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8"
MODEL_LINK = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz"
MODEL_CONFIG = "ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.config"
MODELS_FOLDER = OBJECT_DETECTION_DIRECTORY+"configs/tf2/"
MODEL_FILEPATH = os.path.join(MODELS_FOLDER, MODEL_CONFIG)
MODEL_OUTPUT = DATASET_DIRECTORY + "modelOutput/"

In [4]:
FINE_CHECKPOINT = DATASET_DIRECTORY + "models/" + MODEL_NAME + "/checkpoint/cpkt-0"
TRAIN_TFRECORD = DATASET_DIRECTORY + "train.record"
TEST_TFRECORD = DATASET_DIRECTORY + "test.record"
VALID_TFRECORD = DATASET_DIRECTORY + "valid.record"
LABEL_MAP = DATASET_DIRECTORY + "label_map.pbtxt"

In [5]:
BATCH_SIZE = 12
NUM_STEPS = 10000
NUM_CLASSES = 52
NUM_EVAL_STEPS = 50

## Downloading ML Model

In [ ]:
wget.download(MODEL_LINK, out='./models')

## Unzip MLModel

In [ ]:
tar = tarfile.open('./models/' + MODEL_NAME + ".tar.gz")
tar.extractall('./models/')
tar.close()

## Modify config file


In [6]:
with open(MODEL_FILEPATH) as f:
    s = f.read()

with open(MODEL_FILEPATH, 'w') as f:

    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(FINE_CHECKPOINT), s)

    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(TRAIN_TFRECORD), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(TEST_TFRECORD), s)

    
    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(LABEL_MAP), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(BATCH_SIZE), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(NUM_STEPS), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(NUM_CLASSES), s)

    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: true', 'fine_tune_checkpoint_type: {}'.format('False'), s)
        
    f.write(s)

In [7]:
!cat {MODEL_FILEPATH}

# SSD with Resnet 101 v1 FPN feature extractor, shared box predictor and focal
# loss (a.k.a Retinanet).
# See Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 39.5 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 52
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7

## Training model

In [8]:
!python /tf/tensorflow/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={MODEL_FILEPATH} \
    --model_dir ={MODEL_OUTPUT} \
    --alsologtostderr \
    --num_train_steps={NUM_STEPS} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={NUM_EVAL_STEPS}

2024-05-11 07:01:48.005394: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-11 07:01:48.006688: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 07:01:48.027895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 07:01:48.027945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 07:01:48.028642: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to